In [ ]:
#!/usr/bin/env python
# coding: utf-8

############ Import libraries ############
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import json
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

pd.options.mode.chained_assignment = None  # default='warn'

############ Number of days for prediction ############
daynum = 14

############ Convert saved MISP attribute JSON file to dataframe ############
f = open('MISP_attributes_all.json')
data = json.load(f)
dict = data['response']['Attribute']
df_dict = pd.DataFrame.from_dict(dict)

############ Data prep for all attribution ############
# extract id,event_id,timestamp
df_all = df_dict[['id', 'event_id', 'timestamp']]
# get timestamp and convert unix time to date and drop hours/minutes
df_all = pd.to_datetime(df_all['timestamp'], unit='s').dt.date
df_all = pd.to_datetime(df_all)  # convert object to datetime
df_all = df_all.value_counts().rename_axis('Date').reset_index(name='Num').sort_values(
    by=['Date']).reset_index(drop=True)  # group by date and name headers
df_all = df_all[df_all['Date'] > datetime.now(
) - pd.to_timedelta("1500days")]  # filter last 1500 days
q = df_all.Num.quantile(0.95)  # cut outliers
df_all = df_all.query('Num < @q')

############ Model creation for all attribution ############
# numbers to NumPy
dataset = df_all.Num.values
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))  # 1D to 2D
# create dataset


def gen_dataset(dataset, lag_max, forecast_horizon=1):
    X, y = [], []
    for i in range(len(dataset) - lag_max - forecast_horizon + 1):
        a = i + lag_max
        b = a + forecast_horizon
        X.append(dataset[i:a, 0])  # lag features
        y.append(dataset[a:b, 0])  # objective variable
    return np.array(X), np.array(y)


lag_max = 150  # set dataset length for analysis
forecast_horizon = daynum  # 14days prediction period
X, y = gen_dataset(dataset, lag_max, forecast_horizon)
test_length = 70  # time period of test data
X_train_0 = X[:-test_length, :]  # leaning data
X_test_0 = X[-test_length:, :]  # test data
y_train = y[:-test_length, :]  # learning data
y_test = y[-test_length:, :]  # test data
scaler_y = MinMaxScaler(feature_range=(0, 1))  # normalization（0-1）
y_train = scaler_y.fit_transform(y_train)  # objective variable y
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_0 = scaler_X.fit_transform(X_train_0)  # explanatory variable X
X_test_0 = scaler_X.transform(X_test_0)
# rebuild for model construction (sample number, time steps, features)
X_train = np.reshape(X_train_0, (X_train_0.shape[0], X_train_0.shape[1], 1))
X_test = np.reshape(X_test_0, (X_test_0.shape[0], X_test_0.shape[1], 1))
# settings for each model
models = [
    ['SimpleRNN',
     Sequential([
         SimpleRNN(300, input_shape=(X_train.shape[1], X_train.shape[2])),
         Dropout(0.2),
         Dense(forecast_horizon, activation='linear'),
     ])
     ],
]
for model_name, model in models:
    model.compile(loss='mean_squared_error', optimizer='adam')
    # EaelyStopping setting
    early_stopping = EarlyStopping(monitor='val_loss',
                                   min_delta=0.0,
                                   patience=2)
    # leaning process
    history = model.fit(X_train, y_train,
                        epochs=2000,
                        batch_size=64,
                        validation_split=0.2,
                        callbacks=[early_stopping],
                        verbose=0,
                        shuffle=False)
    # predict objective variable of test data
    y_test_pred = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred)
    # combine test data objective variable and prediction result
    df_test = pd.DataFrame(np.hstack((y_test, y_test_pred)))
    # save Smodel
    filename = model_name + '.h5'
    model.save(filename)
# load model
filename = 'SimpleRNN.h5'
model = load_model(filename)
# predict objective variable of test data
# close to 0 is better
y_test_pred = model.predict(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred)
# accuracy index for 14days prediction
testResult = []
for i in range(forecast_horizon):
    RMSE = np.sqrt(mean_squared_error(y_test[:, i], y_test_pred[:, i]))
    MAE = mean_absolute_error(y_test[:, i], y_test_pred[:, i])
    MAPE = mean_absolute_percentage_error(y_test[:, i], y_test_pred[:, i])
    testResult.append([i+1, RMSE, MAE, MAPE])

testResult = pd.DataFrame(testResult, columns=(
    ['n-step ahead', 'RMSE', 'MAE', 'MAPE']))

############ combine 14 days actual value（y）and predicted value ##############
df_test_last_ALL = pd.DataFrame(y_test[test_length-1], columns=['y'])
df_test_last_ALL['Predict'] = pd.DataFrame(y_test_pred[test_length-1])
predict_ALL = df_test_last_ALL['Predict'].sum()
na_prev = df_test_last_ALL['y'].sum()
accuracy_ALL = (100 - abs((((na_prev - predict_ALL)/na_prev)*100))
                ).round(1).astype(str)+'%'


############ Data prep for each category ############
df_cat = df_dict[['id', 'event_id', 'timestamp', 'category']]
# get timestamp and convert unix time to date and drop hours/minutes
df_cat['timestamp'] = pd.to_datetime(df_cat['timestamp'], unit='s').dt.date
df_cat['timestamp'] = pd.to_datetime(
    df_cat['timestamp'])  # convert object to datetime
df_cat = df_cat.value_counts(['timestamp', 'category']).reset_index(
    name='Num').sort_values(by=['timestamp'])

############ Data prep for External analysis ############
df_cat_EA = df_cat[df_cat['category'].str.contains("External analysis")]
q = df_cat_EA.Num.quantile(0.95)  # cut outliers
df_cat_EA = df_cat_EA.query('Num < @q')

############ Model creation for External analysis ############
dataset = df_cat_EA.Num.values
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))


def gen_dataset(dataset, lag_max, forecast_horizon=1):
    X, y = [], []
    for i in range(len(dataset) - lag_max - forecast_horizon + 1):
        a = i + lag_max
        b = a + forecast_horizon
        X.append(dataset[i:a, 0])
        y.append(dataset[a:b, 0])
    return np.array(X), np.array(y)


lag_max = 150
forecast_horizon = daynum
X, y = gen_dataset(dataset, lag_max, forecast_horizon)
test_length = 70
X_train_0 = X[:-test_length, :]
X_test_0 = X[-test_length:, :]
y_train = y[:-test_length, :]
y_test = y[-test_length:, :]
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train = scaler_y.fit_transform(y_train)
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_0 = scaler_X.fit_transform(X_train_0)
X_test_0 = scaler_X.transform(X_test_0)
X_train = np.reshape(X_train_0, (X_train_0.shape[0], X_train_0.shape[1], 1))
X_test = np.reshape(X_test_0, (X_test_0.shape[0], X_test_0.shape[1], 1))
models = [
    ['SimpleRNN',
     Sequential([
         SimpleRNN(300, input_shape=(X_train.shape[1], X_train.shape[2])),
         Dropout(0.2),
         Dense(forecast_horizon, activation='linear'),
     ])
     ],
]
for model_name, model in models:
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='val_loss',
                                   min_delta=0.0,
                                   patience=2)
    history = model.fit(X_train, y_train,
                        epochs=2000,
                        batch_size=64,
                        validation_split=0.2,
                        callbacks=[early_stopping],
                        verbose=0,
                        shuffle=False)
    y_test_pred = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred)
    df_test = pd.DataFrame(np.hstack((y_test, y_test_pred)))
    filename = model_name + '.h5'
    model.save(filename)

filename = 'SimpleRNN.h5'
model = load_model(filename)
y_test_pred = model.predict(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred)

############ combine 14 days actual value（y）and predicted value ##############
df_test_last_EA = pd.DataFrame(y_test[test_length-1], columns=['y'])
df_test_last_EA['Predict'] = pd.DataFrame(y_test_pred[test_length-1])
predict_EA = df_test_last_EA['Predict'].sum()
na_prev = df_test_last_EA['y'].sum()
accuracy_EA = (100 - abs((((na_prev - predict_EA)/na_prev)*100))
               ).round(1).astype(str)+'%'


############ Data prep for Network activity ############
df_cat_NA = df_cat[df_cat['category'].str.contains("Network activity")]
q = df_cat_NA.Num.quantile(0.95)
df_cat_NA = df_cat_NA.query('Num < @q')

############ Model creation for Network activity ############
dataset = df_cat_NA.Num.values
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))


def gen_dataset(dataset, lag_max, forecast_horizon=1):
    X, y = [], []
    for i in range(len(dataset) - lag_max - forecast_horizon + 1):
        a = i + lag_max
        b = a + forecast_horizon
        X.append(dataset[i:a, 0])
        y.append(dataset[a:b, 0])
    return np.array(X), np.array(y)


lag_max = 150
forecast_horizon = daynum
X, y = gen_dataset(dataset, lag_max, forecast_horizon)
test_length = 70
X_train_0 = X[:-test_length, :]
X_test_0 = X[-test_length:, :]
y_train = y[:-test_length, :]
y_test = y[-test_length:, :]
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train = scaler_y.fit_transform(y_train)
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_0 = scaler_X.fit_transform(X_train_0)
X_test_0 = scaler_X.transform(X_test_0)
X_train = np.reshape(X_train_0, (X_train_0.shape[0], X_train_0.shape[1], 1))
X_test = np.reshape(X_test_0, (X_test_0.shape[0], X_test_0.shape[1], 1))
models = [
    ['SimpleRNN',
     Sequential([
         SimpleRNN(300, input_shape=(X_train.shape[1], X_train.shape[2])),
         Dropout(0.2),
         Dense(forecast_horizon, activation='linear'),
     ])
     ],
]
for model_name, model in models:
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='val_loss',
                                   min_delta=0.0,
                                   patience=2)
    history = model.fit(X_train, y_train,
                        epochs=2000,
                        batch_size=64,
                        validation_split=0.2,
                        callbacks=[early_stopping],
                        verbose=0,
                        shuffle=False)
    y_test_pred = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred)
    df_test = pd.DataFrame(np.hstack((y_test, y_test_pred)))
    filename = model_name + '.h5'
    model.save(filename)

filename = 'SimpleRNN.h5'
model = load_model(filename)
y_test_pred = model.predict(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred)

############ combine 14 days actual value（y）and predicted value ##############
df_test_last_NA = pd.DataFrame(y_test[test_length-1], columns=['y'])
df_test_last_NA['Predict'] = pd.DataFrame(y_test_pred[test_length-1])
predict_NA = df_test_last_NA['Predict'].sum()
na_prev = df_test_last_NA['y'].sum()
accuracy_NA = (100 - abs((((na_prev - predict_NA)/na_prev)*100))
               ).round(1).astype(str)+'%'


############ Data prep for Payload delivery ############
df_cat_PD = df_cat[df_cat['category'].str.contains("Payload delivery")]
q = df_cat_NA.Num.quantile(0.95)
df_cat_PD = df_cat_PD.query('Num < @q')

############ Model creation for Payload delivery ############
dataset = df_cat_PD.Num.values
dataset = dataset.astype('float32')
dataset = np.reshape(dataset, (-1, 1))


def gen_dataset(dataset, lag_max, forecast_horizon=1):
    X, y = [], []
    for i in range(len(dataset) - lag_max - forecast_horizon + 1):
        a = i + lag_max
        b = a + forecast_horizon
        X.append(dataset[i:a, 0])
        y.append(dataset[a:b, 0])
    return np.array(X), np.array(y)


lag_max = 150
forecast_horizon = daynum
X, y = gen_dataset(dataset, lag_max, forecast_horizon)
test_length = 70
X_train_0 = X[:-test_length, :]
X_test_0 = X[-test_length:, :]
y_train = y[:-test_length, :]
y_test = y[-test_length:, :]
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train = scaler_y.fit_transform(y_train)
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_train_0 = scaler_X.fit_transform(X_train_0)
X_test_0 = scaler_X.transform(X_test_0)
X_train = np.reshape(X_train_0, (X_train_0.shape[0], X_train_0.shape[1], 1))
X_test = np.reshape(X_test_0, (X_test_0.shape[0], X_test_0.shape[1], 1))
models = [
    ['SimpleRNN',
     Sequential([
         SimpleRNN(300, input_shape=(X_train.shape[1], X_train.shape[2])),
         Dropout(0.2),
         Dense(forecast_horizon, activation='linear'),
     ])
     ],
]

for model_name, model in models:
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='val_loss',
                                   min_delta=0.0,
                                   patience=2)
    history = model.fit(X_train, y_train,
                        epochs=2000,
                        batch_size=64,
                        validation_split=0.2,
                        callbacks=[early_stopping],
                        verbose=0,
                        shuffle=False)
    y_test_pred = model.predict(X_test)
    y_test_pred = scaler_y.inverse_transform(y_test_pred)
    df_test = pd.DataFrame(np.hstack((y_test, y_test_pred)))
    filename = model_name + '.h5'
    model.save(filename)

filename = 'SimpleRNN.h5'
model = load_model(filename)
y_test_pred = model.predict(X_test)
y_test_pred = scaler_y.inverse_transform(y_test_pred)

############ combine 14 days actual value（y）and predicted value ##############
df_test_last_PD = pd.DataFrame(y_test[test_length-1], columns=['y'])
df_test_last_PD['Predict'] = pd.DataFrame(y_test_pred[test_length-1])
predict_PD = df_test_last_PD['Predict'].sum()
na_prev = df_test_last_PD['y'].sum()
accuracy_PD = (100 - abs((((na_prev - predict_PD)/na_prev)*100))
               ).round(1).astype(str)+'%'


########## Output combined prediction values ############
last_ALL = df_all['Num'].tail(daynum).sum()
last_EA = df_cat_EA['Num'].tail(daynum).sum()
last_NA = df_cat_NA['Num'].tail(daynum).sum()
last_PD = df_cat_PD['Num'].tail(daynum).sum()
pred_value = [[last_ALL, int(predict_ALL), accuracy_ALL], [last_EA, int(predict_EA), accuracy_EA], [
    last_NA, int(predict_NA), accuracy_NA], [last_PD, int(predict_PD), accuracy_PD]]
index = ["All attributes", "External analysis",
         "Network activity", "Payload delivery"]
column = ["numbers_last_14days",
          "predicted_numbers_14days", "prediction_accuracy"]
Prediction_table = pd.DataFrame(data=pred_value, index=index, columns=column).sort_values(
    by=['predicted_numbers_14days'], ascending=False)
Prediction_table['predicted_change'] = Prediction_table[[
    'numbers_last_14days', 'predicted_numbers_14days']].pct_change(axis=1).mul(100).round(1)['predicted_numbers_14days']
Prediction_table['predicted_change'] = Prediction_table['predicted_change'].astype(
    str)+'%'
Prediction_table = Prediction_table[[
    'numbers_last_14days', 'predicted_numbers_14days', 'predicted_change', 'prediction_accuracy']]


######### Export to SQL DB ###########
import os
from sqlalchemy import create_engine
# Database connection credentials
username = os.environ.get('MySQLroot')
password = os.environ.get('MySQLrootpw')
hostname = 'localhost:3306'
dbname = 'securemation'
# Create SQLAlchemy engine to connect to mySQL db
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=hostname, db=dbname, user=username, pw=password))
connection = engine.connect()
Prediction_table.to_sql('misp_prediction', engine,
                        index=False, if_exists='replace')

connetion.close()